In [1]:
import pandas as pd
import datetime
from googlesearch import search
import re
import os
import zipfile
from tld import get_tld

In [2]:
def creation_Dataframe_Ransom(url,delta_Days):
    df = pd.read_json(url)
    now = datetime.datetime.now()
    yesterday_date= datetime.datetime.now() - datetime.timedelta(days=int(delta_Days))
    df = df.loc[df["discovered"].between(str(yesterday_date), str(now))]
    df = df.reset_index(drop=True)
    return df

In [3]:
def url_Creation(dataframe_Ransom,list_Url):
    url_Pattern_No_HTTP = re.compile('^[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)$')
    url_Pattern_With_HTTP = re.compile('^https?:\\/\\/(?:www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)$')
    for Victim in dataframe_Ransom['post_title']:
        if url_Pattern_No_HTTP.match(str(Victim)):
            list_Url.append(Victim)
        elif url_Pattern_With_HTTP.match(str(Victim)):
            list_Url.append(Victim)        
        else:
            for created_Url in search(Victim, tld="com", num=1, stop=1, pause=2):
                list_Url.append(created_Url)   
    dataframe_Ransom["url_of_website"] = pd.Series(list_Url)

In [4]:
def country_Creation(dataframe_Ransom, list_Country):
    tld_df = pd.read_json("tld.json")
    list_tld=[]
    list_Country=[]
    list_sld=['gov.',"edu.","ac.","in.","co.","com.","im."]
    url_Pattern_No_HTTP = re.compile('^[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)$')
    url_Pattern_With_HTTP = re.compile('^https?:\\/\\/(?:www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)$')
    for url_Victim in dataframe_Ransom['url_of_website']:
        if url_Pattern_No_HTTP.match(str(url_Victim)):
            list_tld.append(get_tld(url_Victim, fix_protocol=True))
        elif url_Pattern_With_HTTP.match(str(url_Victim)):
            list_tld.append(get_tld(url_Victim))

    dataframe_Ransom["TLD"] = pd.Series(list_tld)
    for tld in dataframe_Ransom['TLD']:
        for i in list_sld:
            if i in tld:
                tld=tld.replace(i,"")
        for i in range(len(tld_df)):
            if tld == tld_df['tlds'][i]:
                list_Country.append(tld_df['country'][i])
    dataframe_Ransom["Country"] = pd.Series(list_Country)

In [5]:
def output_Creation(dataframe_Ransom):
    var1="Ransomware_"
    date= datetime.datetime.now()
    date_formatted=date.strftime("%d_%m_%Y")
    dataframe_Ransom.to_csv(f'{var1}{date_formatted}.csv')
    dataframe_Ransom.to_excel(f'{var1}{date_formatted}.xlsx')
    Output_Zip = zipfile.ZipFile(f'{var1}{date_formatted}.zip', 'w')
    Output_Zip.write(f'{var1}{date_formatted}.csv')
    Output_Zip.write(f'{var1}{date_formatted}.xlsx')
    Output_Zip.close()
    os.remove(f'{var1}{date_formatted}.xlsx')
    os.remove(f'{var1}{date_formatted}.csv')

In [6]:
def misc():
    if os.path.isfile(".google-cookie"):
        os.remove(".google-cookie")

In [7]:
def main():
    url = "https://raw.githubusercontent.com/joshhighet/ransomwatch/main/posts.json"
    url_List = []
    country_list=[]
    Dataframe_Ransom=creation_Dataframe_Ransom(url,2)
    url_Creation(Dataframe_Ransom,url_List)
    country_Creation(Dataframe_Ransom, country_list)
    output_Creation(Dataframe_Ransom)
    misc()

In [8]:
if __name__ == '__main__':
    main()